
nii 파일로부터 segmentation label image (axial, sagittal) 들을 만든다.
image 들은 0 부터 시작되는데 방향에 따라서 어떤것은 0, 어떤것은 마지막으로 맞춰야 한다.

sagittal 은 방향을 조절하지 않은 그냥 그대로의 순서다.


In [1]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from tqdm import tqdm

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from utils.dcm_utils import *
from utils.nii_utils import *
from utils.train_utils import *
from utils.xray_utils import *

from PIL import Image

In [2]:
DATA_DIR = "/Volumes/SSD970/"

TRAIN_SEG_DIR = os.path.join(DATA_DIR, "segmentations")
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")
SEG_IMAGES_DIR = os.path.join(DATA_DIR, "segmentation_images")
SEG_SAGITTAL_IMAGES_DIR = os.path.join(DATA_DIR, "segmentation_sagittal_images")

In [3]:
patient_id = "1.2.826.0.1.3680043.1363"
imgs = read_patient_nii_mask(patient_id, TRAIN_SEG_DIR)

In [4]:
imgs.shape

(199, 512, 512)

In [5]:
imgs[100, :, :]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
imgs[100, :, :].sum()

11176.0

In [7]:
Image.fromarray(np.uint8(imgs[100, :, :] * 32), 'L').save('sample_segmentation_image.png')

In [8]:
stored_img = np.round(np.asarray(Image.open('sample_segmentation_image.png'), np.uint8) // 32)
stored_img.sum()

11176

In [9]:
(imgs[100, :, :] - stored_img).sum()

0.0

In [10]:
def save_axial_images(patient_id, imgs):
    file_dir = os.path.join(SEG_IMAGES_DIR, patient_id)
    os.mkdir(file_dir)
    for i in range(imgs.shape[0]):
        filepath = os.path.join(file_dir, f'{i}.png')
        Image.fromarray(np.uint8(imgs[i, :, :] * 32), 'L').save(filepath)

In [18]:
def save_sagittal_images(patient_id, imgs):
    file_dir = os.path.join(SEG_SAGITTAL_IMAGES_DIR, patient_id)
    os.mkdir(file_dir)
    for i in range(imgs.shape[2]):
        filepath = os.path.join(file_dir, f'{i}.png')
        Image.fromarray(np.uint8(imgs[:, :, i] * 32), 'L').save(filepath)

In [12]:
nii_files = glob.glob(TRAIN_SEG_DIR + "/**")

In [13]:
nii_path = nii_files[0]
print(nii_path)
UID = nii_path.split("/")[-1].replace(".nii", "")
UID

/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.10633.nii


'1.2.826.0.1.3680043.10633'

In [19]:

for nii_path in tqdm(nii_files):
    seg_mask = nib.load(nii_path).get_fdata()
    seg_mask = np.rot90(seg_mask, axes=(0, 1))
    seg_mask = seg_mask.transpose((2, 0, 1))
    # TODO: incorrect!!! , clip 하는 것이 아니라 7 보다 큰 부위는 짤라버려야 된다.
    seg_mask = np.clip(seg_mask, a_min=0, a_max=7)
    UID = nii_path.split("/")[-1].replace(".nii", "")


100%|██████████| 87/87 [05:18<00:00,  3.67s/it]


6